In [28]:
import sys
sys.path.insert(1, '/home/oneills2/VoidFinder/python/')
sys.path.insert(1, '/Users/hellydouglass/Documents/Reserach/VoidFinder/python/')

In [29]:
# Import the necessary libraries/modules
import numpy as np
import time
import pickle
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # Import the Axes3D module explicitly
from sklearn.neighbors import NearestNeighbors  # Import the NearestNeighbors class explicitly
from astropy.table import Table
from vast.voidfinder.distance import z_to_comoving_dist
from vast.voidfinder import ra_dec_to_xyz
from vast.voidfinder._voidfinder_cython_find_next import MaskChecker

ModuleNotFoundError: No module named 'vast'

In [30]:
def calc_volume_boundaries(void_cat_A, void_cat_B):
    """Compute the boundaries of the minimal rectangular volume (parallelpiped)
    that completely contains two void catalogs.
    
    Parameters
    ----------
    void_cat_A : astropy.Table
        Table of void data from the first catalog.
    void_cat_B : astropy.Table
        Table of void data from the second catalog.
        
    Returns
    -------
    x_min : float
        Minimum x-coordinate of the enclosing volume.
    x_max : float
        Maximum x-coordinate of the enclosing volume.
    y_min : float
        Minimum y-coordinate of the enclosing volume.
    y_max : float
        Maximum y-coordinate of the enclosing volume.
    z_min : float
        Minimum z-coordinate of the enclosing volume.
    z_max : float
        Maximum z-coordinate of the enclosing volume.
    """
    
    # Calculating the minimum and maximum x-coordinates
    x_min = np.minimum(np.min(void_cat_A['x']), np.min(void_cat_B['x']))
    x_max = np.maximum(np.max(void_cat_A['x']), np.max(void_cat_B['x']))
    
    # ' ' y-coordinates
    y_min = np.minimum(np.min(void_cat_A['y']), np.min(void_cat_B['y']))
    y_max = np.maximum(np.max(void_cat_A['y']), np.max(void_cat_B['y']))

    # ' ' z-coordinates
    z_min = np.minimum(np.min(void_cat_A['z']), np.min(void_cat_B['z']))
    z_max = np.maximum(np.max(void_cat_A['z']), np.max(void_cat_B['z']))

    # Returning the boundaries
    return x_min, x_max, y_min, y_max, z_min, z_max

In [31]:
def generate_grid_points(x_min, x_max, y_min, y_max, z_min, z_max):
    """Creates a dense rectangular grid of points in 3D for the void volume calculation.
    
    Parameters
    ----------
    x_min : float
        Minimum x-coordinate of the grid boundaries.
    x_max : float
        Maximum x-coordinate of the grid boundaries.
    y_min : float
        Minimum y-coordinate of the grid boundaries.
    y_max : float
        Maximum y-coordinate of the grid boundaries.
    z_min : float
        Minimum z-coordinate of the grid boundaries.
    z_max : float
        Maximum z-coordinate of the grid boundaries.
    
    Returns
    -------
    point_coords : numpy.ndarray with shape (3, N)
        The grid coordinates where N is the number of grid points.
    """
    
    # Defining the default grid spacing as 1 Megaparsec
    x_range = np.arange(x_min, x_max)
    y_range = np.arange(y_min, y_max)
    z_range = np.arange(z_min, z_max)

    
    # Creating a meshgrid from the input ranges
    X, Y, Z = np.meshgrid(x_range, y_range, z_range)

    x_points = np.ravel(X)
    y_points = np.ravel(Y)
    z_points = np.ravel(Z)
    
    point_coords = np.array([x_points, y_points, z_points])
    
    return point_coords


In [32]:
def mask_point_filter(pts, mask, mask_resolution, rmin, rmax):
    """Determines which grid points generated by generate_grid_points lie in the survey mask
    
    Parameters
    ----------
    pts : numpy.ndarray with shape (3, N)
        The grid points generated by generate_grid_points.
    mask : numpy.ndarray with shape (N, M)
        The survey mask returned by vast.voidfinder.multizmask.generate_mask.
    mask_resolution : int
        The survey mask resolution returned by vast.voidfinder.multizmask.generate_mask.
    rmin : float
        The comoving minimum distance within the survey volume.
    rmax : float
        The comoving maximum distance within the survey volume.
    
    Returns
    -------
    points_in_mask : numpy.ndarray with shape (3, P)
        The list of grid points within the survey mask, where P is the number of points in the mask.
    """
    # Initializing a boolean array to tag points in the mask
    points_boolean = np.ones(pts.shape[1], dtype=bool)
    
    # Declaring a MaskChecker object from VAST
    mask_checker = MaskChecker(0,
                                mask,
                                mask_resolution,
                                rmin,
                                rmax)
    
    # Flagging the points that end up outside the mask
    for i in range(pts.shape[1]):
        # Get the current point
        curr_pt = pts[:, i]
        # Checking if the point is not in the mask
        not_in_mask = mask_checker.not_in_mask(curr_pt)
        # Invert not_in_mask to tag points in the mask
        points_boolean[i] = not bool(not_in_mask)
        
    # Now selecting all points that did not get filtered out during the above process
    points_in_mask = pts[:, points_boolean]
    return points_in_mask


In [33]:
def kd_tree(void_cat):
    """Creates a KD-Tree from the x-y-z coordinates of a void catalog.
    
    Parameters
    ----------
    void_cat : astropy.Table
        The given void catalog, which includes columns 'x', 'y', and 'z'.
    
    Returns
    -------
    sphere_tree : sklearn.neighbors._kd_tree.KDTree
        The KD-Tree object for the void catalog.
    """
    # Extracting x, y, and z coordinates from the void catalog
    cx = void_cat['x']
    cy = void_cat['y']
    cz = void_cat['z']

    # Formatting the void centers in an array of shape (N, 3)
    sphere_coords = np.array([cx, cy, cz]).T

    # Creating a KD-Tree from the coordinates using scikit-learn's neighbors module
    sphere_tree = neighbors.KDTree(sphere_coords)
    
    return sphere_tree


In [34]:
def point_query_VF(point_coords, sphere_tree, void_cat, max_sphere_size):
    """Determines which members of a set of grid points are inside VoidFinder voids
    
    Parameters
    ----------
    point_coords : numpy.ndarray with shape (3, N)
        The grid points within the survey volume.
    sphere_tree : sklearn.neighbors._kd_tree.KDTree
        The KD-Tree object for the void catalog generated by kd_tree.
    void_cat : astropy.Table
        The VoidFinder void catalog, which includes a void radius column 'radius'.
    max_sphere_size : float
        A currently unused parameter corresponding to the maximum sphere size in the void catalog.
    
    Returns
    -------
    true_inside : numpy.ndarray with shape (N,)
        Indicates which grid points are inside voids.
    """
    # Initialize an array that will tag which points are in voids
    true_inside = np.full(point_coords.shape[1], False, dtype=bool)
    
    # k determines the number of nearest neighbors in sphere_tree that are checked for each grid point
    # The default behavior is to check only the single nearest void for void membership.
    # It is possible that more distant voids are very large and enclose a grid point that is not enclosed by
    # its nearest neighbor void, but this source of error is currently not accounted for.
    # Setting k to a higher value will improve this error at the cost of runtime.
    k = min(sphere_tree.data.shape[0], 1)
    
    # Query the sphere tree
    dist, idx = sphere_tree.query(point_coords.T, k=k)
    
    # Checking which points are within void interiors
    interiors = dist < void_cat['radius'][idx]
    
    # Perform an OR operation on the k interiors columns to tag any point inside any of the k voids as belonging to a void
    true_inside += np.any(interiors, axis=1)
    
    return true_inside


In [35]:
def point_query_V2(point_coords, sphere_tree, void_cat):
    """Determines which members of a set of grid points are inside V2 voids
    
    Parameters
    ----------
    point_coords : numpy.ndarray with shape (3, N)
        The grid points within the survey volume.
    sphere_tree : sklearn.neighbors._kd_tree.KDTree
        The KD-Tree object for the void catalog generated by kd_tree.
    void_cat : astropy.Table
        The V2 void catalog which includes a void radius column 'radius'.
    
    Returns
    -------
    true_inside : numpy.ndarray with shape (N,)
        Indicates which grid points are inside voids.
    """
    
    idx = sphere_tree.query(point_coords.T, k=1, return_distance=False)
    
    true_inside = void_cat[idx]['in_void']

    return true_inside


In [36]:
def prep_V2_cat(V2_galzones, V2_zonevoids, data_table_vl):
    """Formates a V2 catalog for use with the void overlap calclator
    
    Parameters
    ----------
    V2_galzones: Astropy Table
        The V2 galzones catalogue
    V2_zonevoids: Astropy Table
        The V2 zonevoids catalogue
    data_table_vl: Astropy Table
        The galaxy catalog used to genreate the V2 void catalog
    Returns
    -------
    V2_galzones : Astropy table
        The V2_galzones object formatted for use with teh volume overlap caculator
    """
    
    # TODO: remove hardcoding of these values and allow user to set them
    omega_M = np.float32(0.3)
    h = np.float32(1.0)
    
    # Index the galaxies in the galaxy catalog
    if ~np.isin("index",data_table_vl.colnames):
        data_table_vl['index'] = np.arange(len(data_table_vl))
    
    # Sort the catalogs
    data_table_vl.sort('index')
    V2_galzones.sort('gal')
    
    # Ensure that each galaxy in the catalog has a zone inthe galzones file
    assert len(data_table_vl) == len(V2_galzones)

    # Modify the galaxy table column names
    # WARNING: don't input a catalog with x-y-z coordinates and no 'redshift' column 
    # TODO: make this warning more clear to user
    if ~np.isin("redshift",data_table_vl.colnames):
        data_table_vl['z'].name = "redshift"
    
    # Calculate galaxy comoving distances
    Rgal = z_to_comoving_dist(data_table_vl['redshift'].astype(np.float32),omega_M,h)
    data_table_vl['Rgal'] = Rgal

    # Select only galaxies with positive redshift 
    z_boolean = data_table_vl['redshift']>0 
    data_table_vl = data_table_vl[z_boolean]
    
    # Calculate the xyz coordinates
    galaxies_xyz = ra_dec_to_xyz(data_table_vl)
    V2_galzones['x'] = galaxies_xyz[:,0]
    V2_galzones['y'] = galaxies_xyz[:,1]
    V2_galzones['z'] = galaxies_xyz[:,2]
    
    # Mark zones that belong to voids
    V2_galzones['in_void'] = V2_zonevoids[V2_galzones['zone']]['void1'] > -1
    
    return V2_galzones

In [37]:
class OverlapCalculator():
    
    def __init__(self, data_table_V1, data_table_V2, title1, title2, mask_file, rmin, rmax, galaxy_table=None, zone_table_V1=None, zone_table_V2=None, use_mask=True, V1_algorithm="VF", V2_algorithm="VF"):
        """Formats a V2 catalog for use with the void overlap calculator

        Parameters
        ----------
        data_table_V1: Astropy Table
            The first void catalog.
        data_table_V2: Astropy Table
            The second void catalog.
        title1: string
            The name of the first void catalog.
        title2: string
            The name of the second void catalog.
        mask_file: string or None
            The path to the survey mask file.
        rmin: float
            The comoving minimum distance within the survey volume.
        rmax: float
            The comoving maximum distance within the survey volume.
        galaxy_table: Astropy Table or None
            The galaxy catalog used to generate the V2 catalogs.
        zone_table_V1: Astropy Table or None
            The zone catalog for the first void catalog.
        zone_table_V2: Astropy Table or None
            The zone catalog for the second void catalog.
        use_mask: bool
            If True, the survey mask is applied. Set to False for a simulation box.
        V1_algorithm: string, either "VF" or "V2"
            Denotes whether VoidFinder or V^2 was used to derive the first void catalog.
        V2_algorithm: string, either "VF" or "V2"
            Denotes whether VoidFinder or V^2 was used to derive the second void catalog.
        """
        # Initialize class variables
        self.max_sphere_size = 0

        # Format V^2 catalogs for ease of use
        if V1_algorithm == "V2":
            data_table_V1 = prep_V2_cat(data_table_V1, zone_table_V1, galaxy_table)
        # Note the maximum sphere size for a VoidFinder catalog
        else:
            self.max_sphere_size = np.max(data_table_V1["radius"])

        # Likewise for the second catalog
        if V2_algorithm == "V2":
            data_table_V2 = prep_V2_cat(data_table_V2, zone_table_V2, galaxy_table)
        else:
            self.max_sphere_size = np.maximum(np.max(data_table_V2["radius"]), self.max_sphere_size)

        # Save information of interest
        self.data_table_V1 = data_table_V1
        self.data_table_V2 = data_table_V2
        self.title1 = title1
        self.title2 = title2
        self.use_mask = use_mask
        self.V1_algorithm = V1_algorithm
        self.V2_algorithm = V2_algorithm

        if use_mask:
            # Load the survey mask from the provided file
            temp_infile = open(mask_file, 'rb')
            self.mask, self.mask_resolution, _ = pickle.load(temp_infile)
            temp_infile.close()

            self.rmin = rmin
            self.rmax = rmax

    def find_overlap(self):
        """Calculates the overlap of two void catalogs

        """
        # Calculate the grid boundaries
        xmin, xmax, ymin, ymax, zmin, zmax = calc_volume_boundaries(self.data_table_V1, self.data_table_V2)

        # Generate the grid
        pts = generate_grid_points(xmin, xmax, ymin, ymax, zmin, zmax)

        # Case of using a survey mask
        if self.use_mask:
            points_in_mask = mask_point_filter(pts, self.mask, self.mask_resolution, self.rmin, self.rmax)
            self.pim = points_in_mask
        # Case of a cubic volume
        else:
            points_in_mask = pts

        (var, self.n_points) = points_in_mask.shape

        # U denotes the number of iterations of the volume overlap algorithm to be performed,
        # with each iteration adding a different random offset to the grid points
        # U is currently set to 1 for testing purposes, but should be set higher for
        # monte carlo sampling (U is the number of random points sampled in each grid cell)
        # TODO: add U as a user input, and change the name of U to something more informative
        U = 1

        # Variables describing the number of points to be found within the void of each catalog
        # for each iteration of the algorithm
        self.count_in_V1 = np.zeros(U)
        self.count_out_V1 = np.zeros(U)

        self.count_in_V2 = np.zeros(U)
        self.count_out_V2 = np.zeros(U)

        self.inside_both = np.zeros(U)
        self.inside_neither = np.zeros(U)
        self.inside_V1 = np.zeros(U)
        self.inside_V2 = np.zeros(U)

        points_in_mask_copy = points_in_mask.copy()

        # Create kd-trees for the void centers
        kdTree_V1 = kd_tree(self.data_table_V1)
        kdTree_V2 = kd_tree(self.data_table_V2)

        # Perform each iteration of the algorithm, with a different random offset delta applied
        # To the grid points
        for i in range(U):
            delta = np.random.rand(3)

            points_in_mask_copy[0] = points_in_mask[0] + delta[0]
            points_in_mask_copy[1] = points_in_mask[1] + delta[1]
            points_in_mask_copy[2] = points_in_mask[2] + delta[2]

            if self.V1_algorithm == "VF":
                true_inside_V1 = point_query_VF(points_in_mask_copy, kdTree_V1, self.data_table_V1, self.max_sphere_size)
            elif self.V1_algorithm == "V2":
                true_inside_V1 = point_query_V2(points_in_mask_copy, kdTree_V1, self.data_table_V1).data.T[0]

            # Number of points inside and outside of catalog A voids
            self.count_in_V1[i] = np.sum(true_inside_V1)
            self.count_out_V1[i] = np.sum(~true_inside_V1)

            if self.V2_algorithm == "VF":
                true_inside_V2 = point_query_VF(points_in_mask_copy, kdTree_V2, self.data_table_V2, self.max_sphere_size)
            elif self.V2_algorithm == "V2":
                true_inside_V2 = point_query_V2(points_in_mask_copy, kdTree_V2, self.data_table_V2).data.T[0]

            # Number of points inside and outside of catalog B voids
            self.count_in_V2[i] = np.sum(true_inside_V2)
            self.count_out_V2[i] = np.sum(~true_inside_V2)

            # Number of points inside both A and B voids
            self.inside_V1_and_V2 = np.logical_and(true_inside_V1, true_inside_V2)
            self.inside_both[i] = np.sum(self.inside_V1_and_V2)

            # Number of points inside neither A nor B voids
            self.not_inside_V1_and_V2 = np.logical_and(~true_inside_V1, ~true_inside_V2)
            self.inside_neither[i] = np.sum(self.not_inside_V1_and_V2)

            # Number of points in A but not B voids
            self.inside_v1 = np.logical_and(true_inside_V1, ~true_inside_V2)
            self.inside_V1[i] = np.sum(self.inside_v1)

            # Number of points not in A but in B voids
            self.inside_v2 = np.logical_and(~true_inside_V1, true_inside_V2)
            self.inside_V2[i] = np.sum(self.inside_v2)

        # The below code is redundant with the above, but assumes U=1
        # TODO: replace this code with something that joins all of the U trials together
        # such as self.inside_V2 = np.sum(self.inside_V2, axis=0) / U, etc.
        # (the divide by U would normalize the sum, I believe axis=0 is the correct choice, but try axis=1 if not)
        self.inside_V1_and_V2 = np.logical_and(true_inside_V1, true_inside_V2)
        self.not_inside_V1_and_V2 = np.logical_and(~true_inside_V1, ~true_inside_V2)
        self.inside_V1 = np.logical_and(true_inside_V1, ~true_inside_V2)
        self.inside_V2 = np.logical_and(~true_inside_V1, true_inside_V2)

        # Volume fractions
        self.r_V1 = self.count_in_V1 / self.n_points
        self.r_V2 = self.count_in_V2 / self.n_points
        self.r_V1_V2 = np.sum(self.inside_V1_and_V2) / self.n_points
        self.r_not_V1_V2 = np.sum(self.not_inside_V1_and_V2) / self.n_points
        self.r_V1_not_V2 = np.sum(self.inside_V1) / self.n_points
        self.r_V2_not_V1 = np.sum(self.inside_V2) / self.n_points

    def plot(self):
        """Plots the overlap of two void catalogs after running find_overlap

        """
        fig = plt.figure()
        plt.pie(
            [self.r_V1_V2, self.r_not_V1_V2, self.r_V2_not_V1, self.r_V1_not_V2],
            labels=[
                f"Common to {self.title1}/{self.title2} Voids",
                "Exterior to All Voids",
                f"Unique to {self.title2} Voids",
                f"Unique to {self.title1} Voids"
            ]
        )

        plt.title(f"Volume Overlap of {self.title1} and {self.title2} Voids")

        return fig

    def report(self):
        """Outputs the overlap of two void catalogs after running find_overlap

        """
        print("Number of points used:", self.n_points)
        print(f"Common to {self.title1}/{self.title2} Voids:", self.r_V1_V2)
        print("Exterior to All Voids:", self.r_not_V1_V2)
        print(f"Unique to {self.title2} Voids:", self.r_V2_not_V1)
        print(f"Unique to {self.title1} Voids:", self.r_V1_not_V2)
        
    def table(self):
        """Returns an Astropy table containing counts of various overlap statistics."""
        data = {
            f'Count in {self.title1}': self.count_in_V1,
            f'Count in {self.title2}': self.count_in_V2,
            'Inside Both': self.inside_V1_and_V2,
            'Not Inside Both': self.not_inside_V1_and_V2,
            f'Inside {self.title1}': self.inside_V1,
            f'Inside {self.title2}': self.inside_V2,
            'Total Points': self.n_points
        }
        return Table(data)
    
    def ratios(self):
        """Returns an Astropy table containing ratios of various overlap statistics."""
        data = {
            f'Ratio {self.title1}': self.r_V1,
            f'Ratio {self.title2}': self.r_V2,
            f'Ratio {self.title1} and {self.title2}': self.r_V1_V2,
            f'Ratio Not {self.title1} and {self.title2}': self.r_not_V1_V2,
            f'Ratio {self.title1} Not {self.title2}': self.r_V1_not_V2,
            f'Ratio {self.title2} Not {self.title1}': self.r_V2_not_V1
        }
        return Table(data)


In [15]:
#Loading VoidFinder void files
vf_1 = Table.read("VoidFinder-nsa_v1_0_1_Planck2018_comoving_holes.txt", format='ascii.commented_header')

In [16]:
#Loading V^2 Revolver void files
v2_1r_g = Table.read("V2_REVOLVER-nsa_v1_0_1_Planck2018_galzones (1).dat", format='ascii.commented_header')
v2_1r_z = Table.read("V2_REVOLVER-nsa_v1_0_1_Planck2018_zobovoids (1).dat", format='ascii.commented_header')

In [18]:
#Loading V^2 VIDE void files
v2_1v_g = Table.read("V2_VIDE-nsa_v1_0_1_Planck2018_galzones.dat", format='ascii.commented_header')
v2_1v_z = Table.read("V2_VIDE-nsa_v1_0_1_Planck2018_zobovoids.dat", format='ascii.commented_header')

In [22]:
#Loading mask file
mask_file = "NSA_main_mask.pickle"

In [24]:
# Comparing VoidFinder with VoidFinder

vf_1_vs_vf_1_planck2018 = OverlapCalculator(vf_1, vf_1, "VF Planck2018", "VF Planck2018(1)", mask_file, 0, 677.42914, galaxy_table=None, 
                                   zone_table_V1 = None, zone_table_V2 = None, V1_algorithm="VF", V2_algorithm="VF")

In [25]:
vf_1_vs_vf_1_planck2018.find_overlap()

NameError: name 'MaskChecker' is not defined

In [23]:
# Comparing V^2 Revolver with V^2 VIDE
v2r_v2v_planck2018 = OverlapCalculator(v2_1r_g, v2_1v_g, "V$^2$ REV.", "V$^2$ VIDE", mask_file, 0, 677.42914, galaxy_table=None, 
                                   zone_table_V1 = v2_1r_z, zone_table_V2 = v2_1v_z, V1_algorithm="V2", V2_algorithm="V2")

AttributeError: 'NoneType' object has no attribute 'colnames'